# Topic Modelling in python - data source is a textfile

In [27]:
# coding: utf-8 
#encoding=utf-8
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
import unicodedata
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
%matplotlib inline
import pyLDAvis
import pyLDAvis.sklearn

### Set working directory ( only if not using a vitual environment)

In [33]:

import os
os.chdir('/home/visa/Topic')

In [3]:
df = pd.DataFrame()
f = open("polarity_pos.txt", "r+", encoding="latin1")
txt = f.readlines()


In [4]:
imd = pd.DataFrame(txt, columns=["comments"])
imd["row"] = imd.index

In [5]:
imd.head()

,comments,row
0,the rock is destined to be the 21st century's ...,0
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic\n,2
3,if you sometimes like to go to the movies to h...,3
4,"emerges as something rare , an issue movie tha...",4


In [6]:
stop_words = nltk.corpus.stopwords.words('english')
extended_stopwords = ['\'ll','\'d','\'m','\'re','\'s','\'ve','ca n\'t','r','n\'t','ca','see','get','movies','movie','go','say','come','many','another','could','would','made','really','want','even','odd','films','plot','ever','actually','also','movie','film']
stops = stop_words + extended_stopwords

#### Check with stemming and lemmatization to clean the data

In [35]:
text = " this is a test for stemmer and stemming "

from nltk.tokenize import word_tokenize
tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
stm = PorterStemmer()
lemm = WordNetLemmatizer()

#tokens = [stm.stem(w) for w in tokens]
tokens = [lemm.lemmatize(w) for w in tokens]
#tokens = lemm.lemmatize(tokens)
print(tokens)

['this', 'is', 'a', 'test', 'for', 'stemmer', 'and', 'stemming']


#### Define function to tokenize and lemmatize the data

In [36]:
def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stm = PorterStemmer()
    lemm = WordNetLemmatizer()
    #tokens = [stm.stem(w) for w in tokens]
    tokens = [lemm.lemmatize(w) for w in tokens]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    #import pdb;pdb.set_trace()
    return filtered_tokens

##### Texblob sentiment analysis

In [39]:
comm = " This is a beautiful phone, good performance" 

comm_blob = TextBlob(comm)

comm_blob.sentiment.polarity


0.7749999999999999

#### Setiment Analysis with textblob 

In [40]:
imd['polarity'] = imd.comments.apply(lambda s: TextBlob(s).sentiment.polarity)

In [42]:
#imd.head()
imd.tail()

,comments,row,polarity
10657,a terrible movie that some people will neverth...,10657,-1.000000
10658,there are many definitions of 'time waster' bu...,10658,0.500000
10659,"as it stands , crocodile hunter has the hurrie...",10659,-0.350000
10660,the thing looks like a made-for-home-video qui...,10660,0.000000
10661,"enigma is well-made , but it's just too dry an...",10661,-0.183333


#### Document term matrix with TF-IDF values 

In [43]:
term_idf_vectorizer       = TfidfVectorizer(max_df=0.99, max_features=2000,min_df=0.005, stop_words=stops, use_idf=True, tokenizer=tokenize, ngram_range=(1,1))
%time term_idf_matrix     = term_idf_vectorizer.fit_transform(imd.comments) 
term_idf_feature_names    = term_idf_vectorizer.get_feature_names()
term_idf_matrix.shape


CPU times: user 4.06 s, sys: 3.55 ms, total: 4.06 s
Wall time: 4.07 s


(10662, 268)

In [44]:
term_idf_feature_names

['acting',
 'action',
 'actor',
 'almost',
 'although',
 'always',
 'american',
 'amusing',
 'anyone',
 'anything',
 'around',
 'art',
 'attempt',
 'audience',
 'away',
 'back',
 'bad',
 'beautiful',
 'beautifully',
 'becomes',
 'best',
 'better',
 'big',
 'bit',
 'book',
 'boring',
 'boy',
 'care',
 'cast',
 'certainly',
 'character',
 'charm',
 'charming',
 'child',
 'cinema',
 'cinematic',
 'classic',
 'clever',
 'comedy',
 'comic',
 'compelling',
 'culture',
 'dark',
 'day',
 'de',
 'debut',
 'despite',
 'dialogue',
 'direction',
 'director',
 'documentary',
 'doe',
 'done',
 'drama',
 'dull',
 'easy',
 'effect',
 'effort',
 'else',
 'emotional',
 'end',
 'ending',
 'engaging',
 'enjoy',
 'enjoyable',
 'enough',
 'entertaining',
 'entertainment',
 'especially',
 'every',
 'everything',
 'exercise',
 'experience',
 'eye',
 'face',
 'fall',
 'familiar',
 'family',
 'fan',
 'far',
 'fascinating',
 'feature',
 'feel',
 'feeling',
 'filmmaker',
 'find',
 'first',
 'flick',
 'full',
 'fu

In [18]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

#### Topic Modelling using LDA 

In [45]:
lda = LatentDirichletAllocation(n_topics=5, max_iter=10,learning_method='online',learning_offset=10.,random_state=1)
%time lda.fit(term_idf_matrix)
print("\nTopics using Latent Dirichlet Allocation model with Term frequencies: \n")
print_top_words(lda, term_idf_feature_names, 10)

/home/visa/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 7.33 s, sys: 0 ns, total: 7.33 s
Wall time: 7.33 s

Topics using Latent Dirichlet Allocation model with Term frequencies: 

Topic #0:
ha comedy work doe much audience way thing thriller le
Topic #1:
good feel never bad time director something better every like
Topic #2:
one performance funny u best well sense make often yet
Topic #3:
little life character drama take like story script big give
Topic #4:
wa love nothing action enough fun kind moment de first



#### Avoid noise in unigrams, TFIDF matrix on bigrams and trigrams

In [46]:
term_idf_vectorizer       = TfidfVectorizer(max_df=0.99, max_features=2000,min_df=0.0005, stop_words=stops, use_idf=True, tokenizer=tokenize, ngram_range=(2,3))
%time term_idf_matrix     = term_idf_vectorizer.fit_transform(imd.comments) 
term_idf_feature_names    = term_idf_vectorizer.get_feature_names()
term_idf_matrix.shape

CPU times: user 4.77 s, sys: 43.7 ms, total: 4.82 s
Wall time: 4.82 s


(10662, 395)

#### Topic Modelling using LDA ( with bigrams and trigams)

In [53]:
lda = LatentDirichletAllocation(n_topics=5, max_iter=10,learning_method='online',learning_offset=10.,random_state=1)
%time lda.fit(term_idf_matrix)
print("\nTopics using Latent Dirichlet Allocation model with Term frequencies: \n")
print_top_words(lda, term_idf_feature_names, 10)

/home/visa/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 5.42 s, sys: 3.89 ms, total: 5.43 s
Wall time: 5.42 s

Topics using Latent Dirichlet Allocation model with Term frequencies: 

Topic #0:
soap opera running time character study one thing sense humor young woman good intention enough make much better ha done
Topic #1:
love story worth seeing blue crush long time stealing harvard doe make woody allen motion picture every bit home video
Topic #2:
romantic comedy play like look like two hour new york make u high school whole lot try hard waste time
Topic #3:
big screen action sequence give u whole thing may find reign fire guilty pleasure doe much seem like human nature
Topic #4:
feel like special effect subject matter never quite good time de niro one best year ago high crime pretty much



In [49]:
pyLDAvis.enable_notebook()

In [54]:
pyLDAvis.sklearn.prepare(lda, term_idf_matrix, term_idf_vectorizer)

/home/visa/.local/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      20.824189        1       1  0.313597  0.146287
4      20.663106        1       2 -0.231561  0.252302
0      20.093874        1       3 -0.038577 -0.222846
3      19.675636        1       4 -0.025500 -0.105461
1      18.743195        1       5 -0.017958 -0.070282, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
104   Default  61.000000         feel like  61.000000  30.0000  30.0000
293   Default  50.000000   romantic comedy  50.000000  29.0000  29.0000
269   Default  33.000000         play like  33.000000  28.0000  28.0000
309   Default  28.000000        soap opera  28.000000  27.0000  27.0000
203   Default  25.000000        love story  25.000000  26.0000  26.0000
200   Default  26.000000         look like  26.000000  25.0000  25.0000
316   Default  25.000000    special effect  25.000000  24.0000  24.0000
331   Default  24.000000    subject matter  24.000000  23.0000  23.0000
294   Default  22.000000      running time  22.000000  22.0000  22.0000
23    Default  21.000000        big screen  21.000000  21.0000  21.0000
360   Default  19.000000          two hour  19.000000  20.0000  20.0000
243   Default  18.000000          new york  18.000000  19.0000  19.0000
240   Default  18.000000       never quite  18.000000  18.0000  18.0000
388   Default  15.000000      worth seeing  15.000000  17.0000  17.0000
127   Default  17.000000         good time  17.000000  16.0000  16.0000
53    Default  16.000000           de niro  16.000000  15.0000  15.0000
1     Default  15.000000   action sequence  15.000000  14.0000  14.0000
38    Default  15.000000   character study  15.000000  13.0000  13.0000
120   Default  15.000000            give u  15.000000  12.0000  12.0000
29    Default  14.000000        blue crush  14.000000  11.0000  11.0000
198   Default  14.000000         long time  14.000000  10.0000  10.0000
252   Default  15.000000          one best  15.000000   9.0000   9.0000
262   Default  14.000000         one thing  14.000000   8.0000   8.0000
211   Default  14.000000            make u  14.000000   7.0000   7.0000
304   Default  14.000000       sense humor  14.000000   6.0000   6.0000
394   Default  13.000000       young woman  13.000000   5.0000   5.0000
376   Default  13.000000       whole thing  13.000000   4.0000   4.0000
322   Default  12.000000  stealing harvard  12.000000   3.0000   3.0000
73    Default  12.000000          doe make  12.000000   2.0000   2.0000
124   Default  12.000000    good intention  12.000000   1.0000   1.0000
...       ...        ...               ...        ...      ...      ...
203    Topic5  25.157205        love story  25.856953   1.6469  -3.1602
388    Topic5  15.260137      worth seeing  15.959640   1.6295  -3.6601
29     Topic5  14.136059        blue crush  14.835557   1.6260  -3.7366
198    Topic5  13.813111         long time  14.513721   1.6249  -3.7597
322    Topic5  11.895397  stealing harvard  12.595903   1.6171  -3.9092
73     Topic5  11.681019          doe make  12.382189   1.6160  -3.9274
379    Topic5  10.624649       woody allen  11.325380   1.6105  -4.0222
229    Topic5  10.364006    motion picture  11.064528   1.6089  -4.0470
89     Topic5  10.329249         every bit  11.029122   1.6088  -4.0504
162    Topic5  10.281313        home video  10.980611   1.6085  -4.0550
232    Topic5  10.270895          much fun  10.971886   1.6083  -4.0560
71     Topic5   9.472188          doe know  10.171476   1.6031  -4.1370
54     Topic5   9.344584          de palma  10.044140   1.6021  -4.1505
28     Topic5   9.118581       blair witch   9.818579   1.6004  -4.1750
70     Topic5   8.990848          doe give   9.692768   1.5992  -4.1891
216    Topic5   8.903339   master disguise   9.604040   1.5986  -4.1989
21     Topic5   8.875733           big fat   9.5752

#### Topic modeling using NMF 

In [55]:
# Fit the NMF model
%time nmf = NMF(n_components=5, random_state=1,alpha=.1, l1_ratio=.5).fit(term_idf_matrix)
print("\nFitting the Non-negative Matrix Factorization model with tf-idf features: \n")
print_top_words(nmf, term_idf_feature_names, 10)

CPU times: user 243 ms, sys: 288 ms, total: 531 ms
Wall time: 200 ms

Fitting the Non-negative Matrix Factorization model with tf-idf features: 

Topic #0:
romantic comedy sandra bullock hugh grant interesting character new york much fun start finish point view might well one greatest
Topic #1:
feel like still feel doe feel make feel one feel like one two hour spy kid tv series three hour
Topic #2:
play like like one like bad big screen young woman seem like whole thing point view never seen after-school special
Topic #3:
love story lan yu story one birthday girl edge seat one best like bad good intention good job good thing
Topic #4:
special effect action sequence jackie chan minority report hollywood ending queen damned time machine funny moment after-school special harry potter

